In [1]:
! pip install -q langchain-google-genai langchain-community langgraph langchain-core

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import os

In [3]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

In [4]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [5]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
model.invoke("Hi").content

'Hi there! How can I help you today?'

In [7]:
messages = [
    ("system", "You are expert in python coding."),
    ("user", "Need python code for factorial of number")
]

In [8]:
print(model.invoke(messages).content)

Several Python functions can calculate the factorial of a number. Here are a few options, each with different strengths and weaknesses:

**Option 1: Iterative Approach (Most Efficient)**

This is generally the most efficient method for calculating factorials, especially for larger numbers:

```python
def factorial_iterative(n):
    """
    Calculates the factorial of a non-negative integer using iteration.

    Args:
        n: The non-negative integer.

    Returns:
        The factorial of n.  Raises ValueError if n is negative or not an integer.
    """
    if not isinstance(n, int):
        raise ValueError("Factorial is only defined for integers.")
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    elif n == 0:
        return 1
    else:
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result

#Example Usage
print(factorial_iterative(5))  # Output: 120
print(factorial_iterative(0))  # Output: 1

#

In [9]:
messages = [
    ("system", "You are an news reporter."),
    ("user", "tell me about 2025 LA wildfire updates")
]

In [10]:
print(model.invoke(messages).content)

**Breaking News:  Increased Fire Risk Prompts LA County to Issue Red Flag Warning**

Los Angeles, CA – October 26, 2024 –  The Los Angeles County Fire Department (LACoFD) has issued a Red Flag Warning effective immediately, citing critically dry conditions, high winds, and low humidity.  This heightened fire danger comes as the region continues to grapple with the ongoing effects of the severe drought.  While no major wildfires are currently burning within LA County, the warning underscores the extreme vulnerability of the region to rapidly spreading wildfires.

Several smaller brush fires have been reported in the Angeles National Forest and the Santa Monica Mountains over the past week, all quickly contained by the swift response of firefighters.  However, officials are urging residents to remain vigilant and take preventative measures.  This includes clearing defensible space around homes, ensuring proper disposal of cigarettes, and avoiding activities that could spark a fire.

“We’

# Tavily Search Tool

In [11]:
search = TavilySearchResults(max_results=2)

In [12]:
search.invoke("tell me about 2025 LA wildfire")

[{'url': 'https://en.wikipedia.org/wiki/January_2025_Southern_California_wildfires',
  'content': 'The January 2025 Southern California wildfires are an ongoing series of wildfires that struck the Los Angeles metropolitan area and surrounding regions on January 7, 2025, and was characterized by meteorological conditions including hurricane-force winds, very low humidity, and prolonged drought conditions that led to extreme fire danger. Vegetation and trees near the Getty Villa burned, with no structural damage being reported as of 5:20 pm PST.[8] The fire also spread to the Palisades Charter Elementary School.[24] Several beachfront properties in Malibu were destroyed by the wildfire.[18] Winds blew wildfire smoke across Los Angeles, leading to several "very unhealthy" air quality index readings of over 200, with the PM2.5 of the Harrison ES station reaching 184.1 µg/m³, or 36.8 times the annual World Health Organization guideline value.[8][28][29] ^ a b c "Live updates: Pacific Palisa

# LangGraph

In [13]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

In [14]:
tools = [search]

In [15]:
agent_executor = create_react_agent(model,tools)

In [16]:
response = agent_executor.invoke({"messages":"tell me about 2025 LA wildfire"})
response

{'messages': [HumanMessage(content='tell me about 2025 LA wildfire', additional_kwargs={}, response_metadata={}, id='0da9d264-6e5a-4464-a433-af6be632f30f'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "2025 LA wildfire"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-74da9b4e-da9c-4235-9631-74ec04c209a1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 LA wildfire'}, 'id': '49a3ebd7-c617-40f1-b110-91eb4c147993', 'type': 'tool

In [17]:
print(response["messages"])

[HumanMessage(content='tell me about 2025 LA wildfire', additional_kwargs={}, response_metadata={}, id='0da9d264-6e5a-4464-a433-af6be632f30f'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "2025 LA wildfire"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-74da9b4e-da9c-4235-9631-74ec04c209a1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 LA wildfire'}, 'id': '49a3ebd7-c617-40f1-b110-91eb4c147993', 'type': 'tool_call'}], usage

# Memory

In [18]:
memory = MemorySaver()

In [19]:
agent_executor = create_react_agent(model,tools,checkpointer=memory)

In [20]:
config = {"configurable":{"thread_id":"001"}}

In [21]:
for chunk in agent_executor.stream({"messages":"tell me about 2025 LA wildfire"},config):
  print(chunk)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "2025 LA wildfire"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-ef6679c6-6401-4b98-a39c-d4405a74c31b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 LA wildfire'}, 'id': 'd447300b-012e-4df1-b4d6-2786c0621ae9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 15, 'total_tokens': 99, 'input_token_details': {'cache_read': 0}})]}}
{'

In [22]:
for chunk in agent_executor.stream({"messages":"How it started"},config):
  print(chunk)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "how did 2025 LA wildfire start"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-ec6cc133-d4bb-4d00-9fe6-632f69ef634c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'how did 2025 LA wildfire start'}, 'id': '544f1765-3a1c-491d-adff-3f2f3df44385', 'type': 'tool_call'}], usage_metadata={'input_tokens': 681, 'output_tokens': 20, 'total_tokens': 701, 'input_token_detai

In [23]:
config = {"configurable":{"thread_id":"002"}}

for chunk in agent_executor.stream({"messages":"How it started"},config):
  print(chunk)

{'agent': {'messages': [AIMessage(content='Could you please provide me with more context? What is the "it" you are referring to?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-e7f28d11-0615-4c26-a181-d047a2d86b5d-0', usage_metadata={'input_tokens': 77, 'output_tokens': 21, 'total_tokens': 98, 'input_token_details': {'cache_read': 0}})]}}
